### 4월 이후를 예측해보기

In [1]:
# 모델 불러오기
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size=6, hidden_size=30, num_layers=2, batch_first=True)
        self.fc = nn.Linear(30, 1)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x[:, -1])
        return x

In [3]:
# Load model
model = RNN()
model.load_state_dict(torch.load('rnn_model.pth'))

<All keys matched successfully>

In [4]:
# 모델 사용하기
# 1. 데이터 로드
import pandas as pd
import numpy as np

data = pd.read_csv('../../DATA/SMP_201004_202403_norm.csv')
data.head()

,날짜,통합,norm,denorm
0,2001-04-01,50.705398,-1.165759,50.705398
1,2001-05-01,52.552364,-1.123687,52.552364
2,2001-06-01,51.247385,-1.153413,51.247385
3,2001-07-01,47.428339,-1.240408,47.428339
4,2001-08-01,42.293200,-1.357382,42.293200


In [5]:
# 마지막 6개월
data_6 = data.iloc[-6:]
data_6

,날짜,통합,norm,denorm
270,2023-10-01,137.01,0.800191,137.01
271,2023-11-01,120.85,0.432079,120.85
272,2023-12-01,131.12,0.666022,131.12
273,2024-01-01,138.06,0.824110,138.06
274,2024-02-01,119.53,0.402011,119.53
275,2024-03-01,134.47,0.742332,134.47


In [18]:
# model prediction
# norm column으로 예측
norm = torch.tensor(data_6['norm'].values).unsqueeze(0).float()
norm, norm.shape


(tensor([[0.8002, 0.4321, 0.6660, 0.8241, 0.4020, 0.7423]]),
 torch.Size([1, 6]))

In [20]:
pred = model(norm.unsqueeze(0))
pred

tensor([[0.4533]], grad_fn=<AddmmBackward0>)

In [21]:
def denorms(x):
    return x * 43.89968 + 101.88185

In [24]:
print('4월 예측 가중평균 SMP :',denorms(pred).item())

4월 예측 가중평균 SMP : 121.78292083740234


실제 데이터는?

In [25]:
# Load data
data_april = pd.read_csv('../../DATA/HOME_전력거래_계통한계가격_시간별SMP_4월_일별.csv', encoding='cp949')
data_april.head()

,기간,01시,02시,03시,04시,05시,06시,07시,08시,09시,...,18시,19시,20시,21시,22시,23시,24시,최대,최소,가중평균
0,2024/04/14,99.52,94.96,86.66,86.65,86.68,100.28,99.37,94.77,95.19,...,118.20,119.00,141.75,139.71,119.84,119.08,98.50,141.75,86.65,106.62
1,2024/04/13,100.36,92.86,86.90,86.90,86.90,100.94,101.13,92.86,96.25,...,97.82,98.06,98.32,98.83,99.09,99.60,111.31,111.31,86.90,96.92
2,2024/04/12,101.85,99.08,87.57,87.34,87.57,95.88,102.01,101.85,137.41,...,97.41,98.42,99.08,99.18,99.44,99.29,99.98,148.26,87.34,109.42
3,2024/04/11,102.01,96.31,86.93,85.92,86.93,96.31,102.32,113.53,158.09,...,128.39,141.14,142.30,141.05,114.74,141.61,116.33,164.20,85.92,123.75
4,2024/04/10,117.35,102.40,95.98,95.95,99.16,149.78,149.95,149.88,110.38,...,110.69,147.73,147.74,147.75,147.74,147.77,142.26,150.28,95.95,129.34


In [27]:
# 가중평균 구하기
means = data_april['가중평균'].mean()
means

121.20928571428574